# Web scrapping

## import modules needded

In [42]:
#import seluruh library yang diperlukan
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd 
import time
import os

## Formating function

In [43]:
def generate_url(search_key):
    link = 'https://kokorojapanstore.com'
    search_key= search_key.replace(' ', '-')
    search_key= search_key.lower()
    link = link+'/collections/'+search_key
    return link

#data cleansing function for price
def convert_price(string_value):
    string_value = string_value.replace("¥", "").replace(",", "")
    float_value = float(string_value)
    return float_value


#data cleansing function for ratings
def convert_rating(string_value):
    float_value = float(string_value)
    return float_value

#data cleansing function for count_review
def convert_review(string_value):
    integer_value = 0
    if(string_value != ""):
        digits = ''.join(filter(str.isdigit, string_value))
        integer_value = int(digits)
    return integer_value

## Function to Load HTML from Web

In [44]:
def load_html(search_key):
    link = generate_url(search_key)

    #setting up to chrome option headless so it will run without GUI
    option = webdriver.ChromeOptions()
    option.add_argument('--headless')

    #Creating chrome driver service
    servis = Service('chromedriver.exe')

    #instantiating chrome driver
    driver = webdriver.Chrome(service=servis, options=option)
    driver.get(link)
    
    #Scrollong down the web page.
    scroll = 1000
    for i in range(1,13):
        end = scroll * i
        command = "window.scrollTo(0,"+str(end)+")"
        driver.execute_script(command)
        time.sleep(1)

    body = driver.page_source
    driver.quit()
    
    return body

## Function to extract information from the HTML

In [45]:
def extract_data(data):
    
    #Extratcting product name rom the HTML tag
    product = data.find('div',class_="product--title").get_text()

    #Extracting normal_price from the HTML tag   
    prices = data.find_all('span',class_="money")
    prices =[price.get_text() for price in prices]
    normal_price = prices[0]
    real_normal_price = convert_price(normal_price)
    real_sale_price = convert_price(normal_price)

    #extracting status of the product from the HTML tag
    status = 'Not sale'
    if(data.find('span').get_text() == 'Sale' or data.find('span').get_text() == 'Sold out'):
        status = data.find('span').get_text()
        
        #extract sale price if the status of product is sale
        if(status == 'Sale'):
            sale_price=prices[1]
            real_sale_price= convert_price(sale_price)
        
            
        
    #Extracting rating of the prodyct from the HTML tag
    rating = data.find('span',class_="stamped-badge").get('data-rating')
    real_rating = convert_rating(rating)
        
    #Extracting rating of the prodyct from the HTML tag
    review = data.find('span',class_="stamped-badge-caption").get_text()
    real_review = convert_review(review)
        
    record = [product, status, real_normal_price,real_sale_price,real_rating,real_review]
        
        
    return record

## Function to create Json File using Pandas data frame

In [46]:
def pandas_to_jsonFile(records):
    coloumns=['Product_name','status','normal_price','sale_price','ratings','count_reviews']
    
    #create dataframe
    df = pd.DataFrame(records, columns= coloumns)
    
    #convert dataframe into json format
    json_data = df.to_json(orient='records',indent=4)
    with open('Kokoro_Best_Sellers.json', 'w',encoding='utf-8') as file:
        file.write(json_data)

## Function to create csv File using Pandas data frame

In [47]:
def pandas_to_csv(records):
    coloumns=['Product_name','status','normal_price','sale_price','ratings','count_reviews']
    
    #create dataframe
    df = pd.DataFrame(records, columns= coloumns)
    
    #convert dataframe into csv
    df.to_csv('Kokoro_Best_sellers.csv',index=False)

## Main Function

In [48]:
def main(search_key):
    records = []
    
    body = load_html(search_key)
    
    #HTML parsing
    soup = BeautifulSoup(body,'html.parser')

    #datas contain all of product
    datas = soup.find_all('article',class_="product--root")
    
    #extractinfg information for each product
    for data in datas:
        record = extract_data(data)
        records.append(record)
    
    
    pandas_to_jsonFile(records)
    
        
    pandas_to_csv(records)
        
    

In [49]:
main('top 100 best sellers')